In [ ]:
import numpy as np

import torch
from torch.autograd import Variable


In [ ]:
batch_size, d = 1024, 5  # d is size of input

x_np = np.random.randint( d, size=(batch_size,1) )

use_cuda = torch.cuda.is_available()

t_long = torch.LongTensor
t_float = torch.FloatTensor
if use_cuda:
    t_long = torch.cuda.LongTensor
    t_float = torch.cuda.FloatTensor

# Create Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable( torch.from_numpy(x_np).type( t_long ) )
y = Variable( torch.from_numpy(x_np).type( t_long ).squeeze(), requires_grad=False)

In [ ]:
class SelectorNet(torch.nn.Module):
  def __init__(self, D):
    """
    In the constructor we instantiate nn modules and assign them as member variables.
    """
    super(SelectorNet, self).__init__()
    
    self.input_space = torch.FloatTensor(batch_size, D).type(t_float) 
    self.linear = torch.nn.Linear(D, D)  # The weights in here should become the identity

  def forward(self, i):
    """
    In the forward function we accept a Variable of input data and we must return
    a Variable of output data. We can use Modules defined in the constructor as
    well as arbitrary operators on Variables.
    """
    
    # Convert the input 'i' into a one-hot vector
    self.input_space.zero_()
    self.input_space.scatter_(1, i.data, 1.)
    #self.input_space.scatter_(1, Variable(i), 1.)
    
    x = Variable( self.input_space )
    #print(x.size())
    logits = self.linear(x)
    #print(logits.size())
    
    #.clamp(min=0)
    #y_pred = self.linear2(h_relu)
    
    #action1 = torch.nn.SoftMax(logits)
    
    action = logits
    #y_probs, y_idx = torch.max(action1)
    return action


In [ ]:
# Construct our model by instantiating the class defined above
model = SelectorNet(d)
if use_cuda: model = model.cuda()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the nn modules 
# which are members of the model.

#criterion = torch.nn.MSELoss(size_average=False)
criterion = torch.nn.NLLLoss(size_average=True)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4) # 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1) # 

for t in range(100):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)

  # Compute and print loss
  loss = criterion(y_pred, y)
  if (t+1) % 10 ==0 :
      print(t+1, loss.data[0])

  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print(model.linear.weight.data,)# model.linear.bias.data)